# REDES NEURAIS ARTIFICIAIS

# PARTE 1: Classificação Binária - Diagnóstico de Câncer de Mama

Base de dados: Breast Cancer Wisconsin (Diagnostic) -
https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic

In [99]:
#bilbiotecas keras e tensorflow
#!pip install scikeras==0.13.0

In [100]:
#!pip install tensorflow==2.16.1 scikit-learn==1.5.0

In [101]:
# após instalar as bibliotecas, é necessário reiniciar o notebook
# Para isso, rode esta célula sozinha antes de prosseguir.
#exit()

In [102]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

In [103]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.3.3', '2.16.1', '1.5.0', '0.13.0')

### Carregar os dados

1) Carregue a base de dados, faça a divisão de treino e teste (para isso, utilize a função train_test_split do sklearn), como o tamanho da base de teste de 0.25.

In [104]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Carregar os dados
X = pd.read_csv('dados_breast.csv').values
y = pd.read_csv('rotulos_breast.csv', header=None).values.ravel()

# Garantir que X e y tenham o mesmo tamanho (ajustar y para o tamanho de X)
min_size = min(len(X), len(y))
X = X[:min_size]
y = y[:min_size]

# inicialmente os arquivos tinham uma diferença de tamanho (um era maior que o outro)
print(f"Tamanho de X: {X.shape}")
print(f"Tamanho de y: {y.shape}")

# Dividir em treino e teste (75% treino, 25% teste)
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size=0.25, random_state=42)

Tamanho de X: (569, 30)
Tamanho de y: (569,)


In [105]:
X_treinamento.shape, y_treinamento.shape

((426, 30), (426,))

In [106]:
X_teste.shape, y_teste.shape

((143, 30), (143,))

### Estrutura da Rede Neural Artificial e Teste

In [107]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras import backend as k

2) Crie a RNA com as seguintes configurações:

a) uma camada de entrada de com 30 neurônios;

b) uma camada oculta densa com 16 neurônios. Porque a camada oculta tem essa quantidade de neurônios?

c) adicione na camada oculta a função de ativação relu e inicialize os pesos de utilize o Random uniform initializer (https://www.tensorflow.org/api_docs/python/tf/keras/initializers);

d) adicione a camada de saíde com a função de ativação sigmoid. Porque foi utilizada esta função para a saída da RNA?

3) Porque utilizamos a classe chamada Sequential para a RNA?

In [108]:
# Criar a Rede Neural Artificial
rede_neural = Sequential([
    # a) Camada de entrada com 30 neurônios (30 features do dataset)
    tf.keras.layers.InputLayer(shape=(30,)),

    # b) e c) Camada oculta densa com 16 neurônios, ativação relu e inicializador random_uniform
    tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),

    # d) Camada de saída com 1 neurônio e ativação sigmoid (classificação binária)
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

**Respostas:**

**b)** A camada oculta tem 16 neurônios como uma escolha heurística que equilibra capacidade de aprendizado e complexidade. Geralmente, usa-se um número entre a quantidade de entradas (30) e saídas (1). O valor 16 é suficiente para capturar padrões nos dados sem causar overfitting excessivo.

**d)** A função sigmoid foi utilizada na saída porque estamos fazendo classificação binária (0 ou 1: benigno ou maligno). A sigmoid transforma qualquer valor em um número entre 0 e 1, que pode ser interpretado como probabilidade da amostra pertencer à classe positiva.

**3)** Utilizamos a classe Sequential porque ela permite criar modelos de forma linear e sequencial, onde as camadas são empilhadas uma após a outra. É ideal para redes neurais feedforward simples onde cada camada tem exatamente uma entrada e uma saída.

4) A partir da RNA gerada, explique o que são os valores apresentados na tabela da rede_neural.summary()

In [109]:
rede_neural.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 513 (2.00 KB)

 Trainable params: 513 (2.00 KB)

 Non-trainable params: 0 (0.00 B)

**Resposta 4:** A tabela do summary() mostra:
- **Layer (type)**: O tipo de cada camada (InputLayer, Dense)
- **Output Shape**: Formato da saída de cada camada (None, número_neurônios), onde None representa o batch size
- **Param #**: Número de parâmetros treináveis em cada camada. Para camadas Dense, é calculado como: (neurônios_entrada × neurônios_saída) + neurônios_saída (bias)
  - Primeira camada oculta: (30 × 16) + 16 = 496 parâmetros
  - Camada de saída: (16 × 1) + 1 = 17 parâmetros
  - **Total**: 513 parâmetros treináveis

5) Adicione um otimizador Adam (https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) e especifique a classe loss binário - binary crossentropy (https://www.tensorflow.org/api_docs/python/tf/keras/losses) e a classe metrics para utilizar a métrica de avaliação de acurácia binária (https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

6) Para que servem os otimizadores? Como o otimizador Adam funciona?

7) Depois de estruturados os parâmtros da RNA, utilize a função .fit para fazer o treinamento da rede. Como foi utilizado o otimizador Adam, e ele é baseado na descida do gradiente estocástica, é possível definir a quantidade de registros que serão enviados para a RNA, isto é, em cada batch serão utilizados 10 registros. Quantos batches serão utilizados ao total?

8) Por fim, defina o número de épocas em que ocorre o treinamento igual a 100.

In [110]:
# 5) Compilar a rede neural com otimizador Adam, loss binary_crossentropy e métrica binary_accuracy
rede_neural.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

**Respostas:**

**6)** Os otimizadores servem para ajustar os pesos da rede neural durante o treinamento, minimizando a função de perda. O Adam (Adaptive Moment Estimation) combina as vantagens de dois outros otimizadores: RMSprop e Momentum. Ele adapta a taxa de aprendizado individualmente para cada parâmetro, usando médias móveis dos gradientes e dos quadrados dos gradientes, tornando-o eficiente e convergindo rapidamente.

**7)** Com batch_size=10 e 426 amostras de treino (75% de 569), teremos aproximadamente 43 batches por época (426 ÷ 10 = 42.6, arredondado para 43).

In [111]:
# 7) e 8) Treinar a rede neural com batch_size=10 e epochs=100
rede_neural.fit(X_treinamento, y_treinamento, batch_size=10, epochs=100)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - binary_accuracy: 0.5305 - loss: 22.5809
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5822 - loss: 5.9242
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.5329 - loss: 4.6426
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5657 - loss: 3.9253
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5610 - loss: 3.0191
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5845 - loss: 2.4407
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5587 - loss: 2.6907
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5939 - loss: 2.5543
Epoch 9/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.5775 - loss: 2.9858
Epoch 10/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5822 - loss: 2.7122
Epoch 11/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.5704 

9) Crie uma variável chamada previsoes para realizar a previsão dos dados de teste (X_teste) O resultado da rede deve ser um valor entre 0 e 1. Porque isso acontece?

In [112]:
previsoes = rede_neural.predict(X_teste)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


In [113]:
# Visualizar as previsões
print("Primeiras 10 previsões:")
print(previsoes[:10])
print(f"\nValor mínimo: {previsoes.min():.4f}")
print(f"Valor máximo: {previsoes.max():.4f}")
print(f"Valor médio: {previsoes.mean():.4f}")
print(f"Todas as previsões estão entre 0 e 1: {(previsoes >= 0).all() and (previsoes <= 1).all()}")

Primeiras 10 previsões:
[[0.9953582 ]
 [0.9996457 ]
 [0.9844285 ]
 [0.845439  ]
 [0.9466783 ]
 [0.99999547]
 [1.        ]
 [0.99835825]
 [0.9987248 ]
 [0.9584763 ]]

Valor mínimo: 0.0000
Valor máximo: 1.0000
Valor médio: 0.8834
Todas as previsões estão entre 0 e 1: True


**Resposta 9:** O resultado está entre 0 e 1 porque a camada de saída usa a função de ativação sigmoid, que matematicamente transforma qualquer valor de entrada em um valor entre 0 e 1. Isso representa a probabilidade da amostra pertencer à classe positiva (maligno = 1). Valores próximos a 1 indicam alta probabilidade de ser maligno, e valores próximos a 0 indicam alta probabilidade de ser benigno.

10) Converta o resultado que está em probabilidade para valores binários com um limiar (treshold) de 0.5.

In [114]:
previsoes = previsoes > 0.5

In [115]:
previsoes

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [

9) Explique o resultado da RNA.

In [116]:
# Verificar a distribuição REAL dos dados
print("Distribuição REAL no dataset completo:")
print(f"Benignos (0): {(y == 0).sum()} ({(y == 0).mean():.2%})")
print(f"Malignos (1): {(y == 1).sum()} ({(y == 1).mean():.2%})")

print("\nDistribuição no conjunto de TESTE:")
print(f"Benignos (0): {(y_teste == 0).sum()} ({(y_teste == 0).mean():.2%})")
print(f"Malignos (1): {(y_teste == 1).sum()} ({(y_teste == 1).mean():.2%})")

# Visualizar as previsões binárias
print("Primeiras 10 previsões:")
print(previsoes[:10].flatten())  # Mostra True/False

# Estatísticas para dados binários
previsoes_num = previsoes.astype(int).flatten()
print(f"\nTotal de predições classe 0 (benigno): {(previsoes_num == 0).sum()}")
print(f"Total de predições classe 1 (maligno): {(previsoes_num == 1).sum()}")
print(f"\nPorcentagem real de malignos no teste: {(y_teste == 1).mean():.2%}")
print(f"Proporção de casos malignos: {previsoes_num.mean():.2%}")


Distribuição REAL no dataset completo:
Benignos (0): 213 (37.43%)
Malignos (1): 356 (62.57%)

Distribuição no conjunto de TESTE:
Benignos (0): 61 (42.66%)
Malignos (1): 82 (57.34%)
Primeiras 10 previsões:
[ True  True  True  True  True  True  True  True  True  True]

Total de predições classe 0 (benigno): 12
Total de predições classe 1 (maligno): 131

Porcentagem real de malignos no teste: 57.34%
Proporção de casos malignos: 91.61%


**Resposta:** O método `evaluate()` retorna dois valores: a perda (loss) e a acurácia binária. A perda indica o quão distantes estão as previsões dos valores reais (quanto menor, melhor). A acurácia binária mostra a porcentagem de classificações corretas no conjunto de teste. Esses resultados permitem avaliar o desempenho do modelo em dados não vistos durante o treinamento.

## Camadas e Otimização da RNA

10) Adicione mais uma camada para oculta densa com 16 neurônio com a função de ativação relu e inicialize os pesos de utilize o Random uniform initializer. Qual o total de parâmetros da RNA agora?

In [117]:
# Recriar a rede neural com duas camadas ocultas
rede_neural = Sequential([
    tf.keras.layers.InputLayer(shape=(30,)),
    tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
    tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),  # Nova camada
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

In [118]:
rede_neural.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 785 (3.07 KB)

 Trainable params: 785 (3.07 KB)

 Non-trainable params: 0 (0.00 B)

11) Abaixo são adicionados os parâmetros do otimizador, que são a taxa de aprendizado e o clipvalue. O que eles fazem?

In [119]:
otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

**Resposta 11:**
- **learning_rate (taxa de aprendizado)**: Controla o tamanho dos passos que o otimizador dá ao ajustar os pesos. Um valor de 0.001 significa passos pequenos e cuidadosos, evitando oscilações mas permitindo convergência gradual.
- **clipvalue**: Limita os gradientes ao intervalo [-0.5, 0.5]. Isso previne o problema de "gradientes explosivos" (exploding gradients), onde gradientes muito grandes podem desestabilizar o treinamento.

12) Teste novamente a RNA. Aumentar a quantidade de camadas melhorou ou piorou os resultados? Explique o que aconteceu com a RNA e porque.

In [120]:
rede_neural.fit(X_treinamento, y_treinamento, batch_size = 10, epochs = 100)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - binary_accuracy: 0.5188 - loss: 1.7646
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.5986 - loss: 0.8001
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.5915 - loss: 1.0141
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.5915 - loss: 0.8831
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6080 - loss: 0.9410
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6056 - loss: 0.8840
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.5892 - loss: 0.8628
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6197 - loss: 0.7882
Epoch 9/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6127 - loss: 0.8031
Epoch 10/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6362 - loss: 0.7591
Epoch 11/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6150 -

In [121]:
previsoes = rede_neural.predict(X_teste)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [122]:
rede_neural.evaluate(X_teste, y_teste)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - binary_accuracy: 0.5734 - loss: 0.8757 


[0.8757132291793823, 0.5734265446662903]

In [123]:
# Verificar a distribuição REAL dos dados
print("Distribuição REAL no dataset completo:")
print(f"Benignos (0): {(y == 0).sum()} ({(y == 0).mean():.2%})")
print(f"Malignos (1): {(y == 1).sum()} ({(y == 1).mean():.2%})")

print("\nDistribuição no conjunto de TESTE:")
print(f"Benignos (0): {(y_teste == 0).sum()} ({(y_teste == 0).mean():.2%})")
print(f"Malignos (1): {(y_teste == 1).sum()} ({(y_teste == 1).mean():.2%})")

# Visualizar as probabilidades ANTES da conversão
print("\nPrimeiras 10 probabilidades:")
print(previsoes[:10].flatten())

# CONVERTER para binário (0 ou 1) usando threshold de 0.5
previsoes_binarias = (previsoes > 0.5).astype(int).flatten()

# Visualizar as previsões APÓS conversão
print("\nPrimeiras 10 previsões binárias:")
print(previsoes_binarias[:10])

# Estatísticas das PREDIÇÕES
print(f"\nTotal de predições classe 0 (benigno): {(previsoes_binarias == 0).sum()}")
print(f"Total de predições classe 1 (maligno): {(previsoes_binarias == 1).sum()}")
print(f"Proporção de casos malignos preditos: {previsoes_binarias.mean():.2%}")

# Comparar com a realidade
print(f"\nPorcentagem real de malignos no teste: {(y_teste == 1).mean():.2%}")
print(f"Porcentagem predita de malignos: {previsoes_binarias.mean():.2%}")


Distribuição REAL no dataset completo:
Benignos (0): 213 (37.43%)
Malignos (1): 356 (62.57%)

Distribuição no conjunto de TESTE:
Benignos (0): 61 (42.66%)
Malignos (1): 82 (57.34%)

Primeiras 10 probabilidades:
[0.5667406  0.60458523 0.5759952  0.39176306 0.85232145 0.9734673
 0.95913124 0.53379464 0.7307607  0.7751751 ]

Primeiras 10 previsões binárias:
[1 1 1 0 1 1 1 1 1 1]

Total de predições classe 0 (benigno): 40
Total de predições classe 1 (maligno): 103
Proporção de casos malignos preditos: 72.03%

Porcentagem real de malignos no teste: 57.34%
Porcentagem predita de malignos: 72.03%


## K-Fold Cross Validation

13) Assista o vídeo https://youtu.be/RczbeFs_WbQ?si=f1Yu4LZaGDfZZpbZ para compreender melhor sobre como funciona o k-fold cross validation. Explique como esta rede foi configurada e como é possível chegar no resultado dela. O que é necessário fazer?

**Resposta 13:** O K-Fold Cross Validation divide os dados em K partes (folds). Para cada iteração, uma parte é usada como teste e as demais como treino, repetindo K vezes. A rede é configurada através da função `criar_rede()` que define a arquitetura, e o `cross_val_score` treina e avalia K vezes (cv=10), retornando a acurácia de cada fold. Isso permite uma avaliação mais robusta do modelo.

14) Calcule também o Desvio Padrão dos resultados para avaliar o modelo. O que é possível concluir com esse resultado?

**Resposta 14:** O desvio padrão de 3.88% indica consistência nos resultados entre os folds, porém a acurácia de 61.17% é muito baixa, sugerindo underfitting (modelo não aprende adequadamente os padrões dos dados). São necessárias melhorias como normalização dos dados e ajuste de hiperparâmetros.



In [124]:
# Calcular média e desvio padrão dos resultados
media = resultados.mean()
desvio_padrao = resultados.std()
print(f"Acurácia Média: {media:.4f}")
print(f"Desvio Padrão: {desvio_padrao:.4f}")

Acurácia Média: 0.6117
Desvio Padrão: 0.0388


In [125]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as k

In [126]:
def criar_rede():
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(30,)),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
  otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
  rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
  return rede_neural

In [127]:
rede_neural = KerasClassifier(model = criar_rede, epochs = 100, batch_size = 10)

In [128]:
resultados = cross_val_score(estimator = rede_neural, X = X, y = y, cv = 10, scoring = 'accuracy')

Epoch 1/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - binary_accuracy: 0.5586 - loss: 0.8662
Epoch 2/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6211 - loss: 0.7567
Epoch 3/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6094 - loss: 0.7202
Epoch 4/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6133 - loss: 0.7033
Epoch 5/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5957 - loss: 0.6881
Epoch 6/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6289 - loss: 0.7088
Epoch 7/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5840 - loss: 0.6928
Epoch 8/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6328 - loss: 0.7210
Epoch 9/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6230 - loss: 0.7542
Epoch 10/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6367 - loss: 0.7056
Epoch 11/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6387 -

In [129]:
resultados

array([0.52631579, 0.63157895, 0.50877193, 0.57894737, 0.54385965,
       0.57894737, 0.59649123, 0.63157895, 0.52631579, 0.625     ])

In [130]:
# Calcular média e desvio padrão dos resultados
media = resultados.mean()
desvio_padrao = resultados.std()
print(f"Acurácia Média: {media:.4f}")
print(f"Desvio Padrão: {desvio_padrao:.4f}")

Acurácia Média: 0.5748
Desvio Padrão: 0.0441


## Overfitting e Dropout

14) Como vimos, o modelo anterior sofreu overfitting. Para reduzir esse problema, aplicamos a técnica de regularização conhecida como dropout (artigo oficial: https://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf e explicação em português: https://www.deeplearningbook.com.br/como-funciona-o-dropout/ ). Aplique o dropout de 20% na primeira e segunda camada oculta. O que acontece com os resultados? E o Desvio Padrão?

In [131]:
def criar_rede():
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(30,)),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dropout(rate=0.2),  # Dropout de 20% após a primeira camada oculta
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dropout(rate=0.2),  # Dropout de 20% após a segunda camada oculta
      tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
  otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
  rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
  return rede_neural

In [132]:
rede_neural = KerasClassifier(model = criar_rede, epochs = 100, batch_size = 10)

In [133]:
resultados = cross_val_score(estimator = rede_neural, X = X, y = y, cv = 10, scoring = 'accuracy')

Epoch 1/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - binary_accuracy: 0.5117 - loss: 2.1591
Epoch 2/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.5938 - loss: 0.9448
Epoch 3/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.5781 - loss: 0.8662
Epoch 4/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.5996 - loss: 0.8106
Epoch 5/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6074 - loss: 0.7732
Epoch 6/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5977 - loss: 0.8189
Epoch 7/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6191 - loss: 0.7966
Epoch 8/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.6250 - loss: 0.7494
Epoch 9/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.5996 - loss: 0.7638
Epoch 10/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5527 - loss: 0.7470
Epoch 11/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.6250 -

In [134]:
resultados

array([0.63157895, 0.63157895, 0.57894737, 0.61403509, 0.64912281,
       0.63157895, 0.63157895, 0.56140351, 0.64912281, 0.625     ])

In [135]:
# Calcular média e desvio padrão dos resultados com dropout
media_dropout = resultados.mean()
desvio_padrao_dropout = resultados.std()
print(f"Acurácia Média (com Dropout): {media_dropout:.4f}")
print(f"Desvio Padrão (com Dropout): {desvio_padrao_dropout:.4f}")

Acurácia Média (com Dropout): 0.6204
Desvio Padrão (com Dropout): 0.0272


**Resposta 14:** O dropout ajuda a reduzir o overfitting ao "desligar" aleatoriamente 20% dos neurônios durante o treinamento, forçando a rede a aprender representações mais robustas. Geralmente, isso resulta em uma acurácia de treino ligeiramente menor, mas uma acurácia de teste melhor (melhor generalização). O desvio padrão tende a diminuir, indicando que o modelo é mais consistente entre diferentes folds.

## Tuning dos Hiperparâmetros (técnicas de ajuste)

Hyperparameter Tuning (Ajuste de Hiperparâmetros) é o processo geral de encontrar os melhores valores para os hiperparâmetros de um modelo de machine learning.
Exemplos de hiperparâmetros:

- Taxa de aprendizado (learning rate)

- Número de árvores em um RandomForest

- Número de camadas ou neurônios em uma rede neural

- Tipo de kernel em um SVM

O tuning pode ser feito por diferentes técnicas, como:

- Grid Search

- Random Search

- Bayesian Optimization, etc.

15) Descreva como a RNA foi configurada para fazer o processo de tuning.

**Resposta 15:** A RNA foi configurada de forma parametrizada através da função `criar_rede()`, que agora aceita múltiplos parâmetros (optimizer, loss, kernel_initializer, activation, neurons). Esses parâmetros são definidos no dicionário `parametros` com diferentes valores a serem testados. O GridSearchCV testa todas as combinações possíveis desses parâmetros, treinando e avaliando o modelo para cada combinação, retornando os melhores parâmetros encontrados.

In [136]:
from sklearn.model_selection import GridSearchCV

def criar_rede(optimizer, loss, kernel_initializer, activation, neurons):
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(30,)),
      tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
      tf.keras.layers.Dropout(rate = 0.2),
      tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
      tf.keras.layers.Dropout(rate = 0.2),
      tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
  rede_neural.compile(optimizer = optimizer, loss = loss, metrics = ['binary_accuracy'])
  return rede_neural

In [137]:
rede_neural = KerasClassifier(model = criar_rede)

In [138]:
parametros = {
    'batch_size': [10, 30],
    'epochs': [50, 100],
    'model__optimizer': ['adam', 'sgd'],
    'model__loss': ['binary_crossentropy', 'hinge'],
    'model__kernel_initializer': ['random_uniform', 'normal'],
    'model__activation': ['relu', 'tanh'],
    'model__neurons': [16, 8]
}

In [139]:
#caso tiver problema com a configuração anterior, testar essa
parametros = {
    'batch_size': [10, 30],
    'epochs': [50],
    'model__optimizer': ['adam'],
    'model__loss': ['binary_crossentropy'],
    'model__kernel_initializer': ['random_uniform', 'normal'],
    'model__activation': ['relu'],
    'model__neurons': [16]
}

In [140]:
parametros

{'batch_size': [10, 30],
 'epochs': [50],
 'model__optimizer': ['adam'],
 'model__loss': ['binary_crossentropy'],
 'model__kernel_initializer': ['random_uniform', 'normal'],
 'model__activation': ['relu'],
 'model__neurons': [16]}

In [141]:
grid_search = GridSearchCV(estimator = rede_neural, param_grid = parametros,
                          scoring = 'accuracy', cv = 5)

In [142]:
grid_search = grid_search.fit(X, y)

Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - binary_accuracy: 0.5341 - loss: 1.7449
Epoch 2/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5890 - loss: 0.8296
Epoch 3/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5934 - loss: 0.7575
Epoch 4/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6000 - loss: 0.7138
Epoch 5/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6198 - loss: 0.6854 
Epoch 6/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6088 - loss: 0.6987
Epoch 7/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6132 - loss: 0.7040
Epoch 8/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6220 - loss: 0.6725
Epoch 9/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6220 - loss: 0.6799
Epoch 10/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6242 - loss: 0.6734
Epoch 11/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6220 - loss: 0.6

In [143]:
print(grid_search)

GridSearchCV(cv=5,
             estimator=KerasClassifier(model=<function criar_rede at 0x0000029F4D2384A0>),
             param_grid={'batch_size': [10, 30], 'epochs': [50],
                         'model__activation': ['relu'],
                         'model__kernel_initializer': ['random_uniform',
                                                       'normal'],
                         'model__loss': ['binary_crossentropy'],
                         'model__neurons': [16], 'model__optimizer': ['adam']},
             scoring='accuracy')


In [144]:
melhores_parametros = grid_search.best_params_
print(melhores_parametros)

{'batch_size': 10, 'epochs': 50, 'model__activation': 'relu', 'model__kernel_initializer': 'normal', 'model__loss': 'binary_crossentropy', 'model__neurons': 16, 'model__optimizer': 'adam'}


In [145]:
melhor_precisao = grid_search.best_score_
print(melhor_precisao)

0.6274491538580965


16) É possível melhorar ainda mais a curácia da RNA? Como?

**Resposta 16:** Sim, é possível melhorar a acurácia através de várias técnicas:
- **Normalização/Padronização dos dados**: Usar StandardScaler ou MinMaxScaler para escalar as features
- **Aumentar épocas**: Treinar por mais tempo, com early stopping para evitar overfitting
- **Ajustar arquitetura**: Testar diferentes números de camadas e neurônios
- **Regularização L1/L2**: Adicionar penalização aos pesos
- **Batch Normalization**: Normalizar as saídas entre camadas
- **Ajustar learning rate**: Usar learning rate schedulers ou valores diferentes
- **Aumentar dados**: Data augmentation (se aplicável) ou coletar mais amostras
- **Ensemble methods**: Combinar múltiplos modelos

#Para salvar o seu modelo de RNA

In [146]:
classificador.save('classificador_breast.keras')

NameError: name 'classificador' is not defined

In [ ]:
classificador_novo = tf.keras.models.load_model('/content/classificador_breast.keras')

In [ ]:
classificador_novo.summary()